<a href="https://colab.research.google.com/github/dtabuena/Resources/blob/main/Tools/Scroll_Plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
def scroll_plot(trace,time,sample_rate,wrap_t,t_range=None,row_dim=[5,0.25],ylims=None,color='k',fig_ax=None,yticks=[0],scale=True,title=None,zmax=None):
    if t_range is not None:
        in_range = np.logical_and( time>=np.min(t_range), time<np.max(t_range) )
        trace = trace[in_range]
        time = time[in_range]
    if ylims is None:
        ylims = [np.min(trace),np.max(trace)]
    if zmax is not None:
        max_zv = (zmax * np.std(trace)) + np.mean(trace)


    num_samples = len(time)
    len_wrap = int(wrap_t*sample_rate)
    num_rows = int(np.ceil(num_samples/len_wrap))
    wrapped_trace = np.full((num_rows, len_wrap), np.nan)
    wrapped_time = np.full((num_rows, len_wrap), np.nan)
    wrapped_trace.flat[:len(trace)] = trace
    wrapped_time.flat[:len(time)] = time
    wrapped_time.flat[len(time)+1:] = time[-1]+np.arange(np.sum(np.isnan(wrapped_time)))*sample_rate

    if fig_ax is None:
        fig_dim = np.array(row_dim)*np.array((1,num_rows))
        fig,ax=plt.subplots(num_rows,1,figsize=fig_dim)
    else:
        [fig,ax] = fig_ax
    for r in range(num_rows):
        if np.max(np.abs(trace)) > max_zv:
              ax[r].set_ylim([-max_zv,max_zv] )
        ax[r].plot(wrapped_time[r,:],wrapped_trace[r,:],color=color)
        ax[r].set_position([0,1-r/num_rows,1,1/num_rows])
        ax[r].set_ylim(ylims)
        ax[r].axis('off')
        ax[r].set_xlim([wrapped_time[r,0],wrapped_time[r,0]+wrap_t])
        for y in yticks:
            wid = plt.rcParams['lines.linewidth']/2
            ax[r].axhline(y,color='k',linewidth=wid,linestyle='dotted')


    if title is not None:
        fig.suptitle(title,color='k')
        fig.subplots_adjust(top=0.9)

    if scale:
        y_scale = log_round(np.diff(ylims)[0]/4)
        x_scale = wrap_t*.05
        x_0 = time[0]

        x0 = time[0] +x_scale
        x1 = time[0]+x_scale +x_scale
        y0 = -y_scale - y_scale*.5
        y1 = 0 - y_scale*.5

        scale_color = 'r'
        ax[0].plot([x0,x1],[y0,y0],color=scale_color)
        ax[0].plot([x0,x0],[y0,y1],color=scale_color)
        ax[0].text( (x0+x1)/2 , y0, str(x_scale), color=scale_color,ha='center',va='top')
        ax[0].text(x0 , (y0+y1)/2 , str(y_scale), color=scale_color,ha='right',va='center')

    plt.show()
    return fig, ax

def log_round(x):
    lg = np.log10(x)
    lg_flr = np.floor(lg)
    x_rnd = np.round(x,int(-lg_flr))
    if lg>1:
        x_rnd=int(x_rnd)
    return x_rnd

"""
_ = scroll_plot(raw_lfp[:,0],time,sample_rate,wrap_t=1,t_range=[0,2],title='Title',zmax=3)
"""

In [ ]:
def scroll_plot(trace,time,sample_rate,wrap_t,t_range=None,row_dim=[5,0.25],ylims=None,color='k',fig_ax=None,yticks=[0],scale=True,title=None,zmax=None):
    if t_range is not None:
        in_range = np.logical_and( time>=np.min(t_range), time<np.max(t_range) )
        trace = trace[in_range]
        time = time[in_range]
    if ylims is None:
        ylims = [np.min(trace),np.max(trace)]
    if zmax is not None:
        max_zv = (zmax * np.std(trace)) + np.mean(trace)


    num_samples = len(time)
    len_wrap = int(wrap_t*sample_rate)
    num_rows = int(np.ceil(num_samples/len_wrap))
    wrapped_trace = np.full((num_rows, len_wrap), np.nan)
    wrapped_time = np.full((num_rows, len_wrap), np.nan)
    wrapped_trace.flat[:len(trace)] = trace
    wrapped_time.flat[:len(time)] = time
    wrapped_time.flat[len(time)+1:] = time[-1]+np.arange(np.sum(np.isnan(wrapped_time)))*sample_rate

    if fig_ax is None:
        fig_dim = np.array(row_dim)*np.array((1,num_rows))
        fig,ax=plt.subplots(num_rows,1,figsize=fig_dim)
    else:
        [fig,ax] = fig_ax
    for r in range(num_rows):
        if np.max(np.abs(trace)) > max_zv:
              ax[r].set_ylim([-max_zv,max_zv] )
        ax[r].plot(wrapped_time[r,:],wrapped_trace[r,:],color=color)
        ax[r].set_position([0,1-r/num_rows,1,1/num_rows])
        ax[r].set_ylim(ylims)
        ax[r].axis('off')
        ax[r].set_xlim([wrapped_time[r,0],wrapped_time[r,0]+wrap_t])
        for y in yticks:
            wid = plt.rcParams['lines.linewidth']/2
            ax[r].axhline(y,color='k',linewidth=wid,linestyle='dotted')


    if title is not None:
        fig.suptitle(title,color='k')
        fig.subplots_adjust(top=0.9)

    if scale:
        y_scale = log_round(np.diff(ylims)[0]/4)
        x_scale = wrap_t*.05
        x_0 = time[0]

        x0 = time[0] +x_scale
        x1 = time[0]+x_scale +x_scale
        y0 = -y_scale - y_scale*.5
        y1 = 0 - y_scale*.5

        scale_color = 'r'
        ax[0].plot([x0,x1],[y0,y0],color=scale_color)
        ax[0].plot([x0,x0],[y0,y1],color=scale_color)
        ax[0].text( (x0+x1)/2 , y0, str(x_scale), color=scale_color,ha='center',va='top')
        ax[0].text(x0 , (y0+y1)/2 , str(y_scale), color=scale_color,ha='right',va='center')

    plt.show()
    return fig, ax

def log_round(x):
    lg = np.log10(x)
    lg_flr = np.floor(lg)
    x_rnd = np.round(x,int(-lg_flr))
    if lg>1:
        x_rnd=int(x_rnd)
    return x_rnd

"""
_ = scroll_plot(raw_lfp[:,0],time,sample_rate,wrap_t=1,t_range=[0,2],title='Title',zmax=3)
"""